In [32]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os, sys, cv2, random, shutil
import pandas as pd
import seaborn as sns
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from mpl_toolkits.axes_grid1 import ImageGrid

from coco_dataset_modules import COCO_dataset
from coco_dataset_modules.visualizer_matplot import VisConfig

def show_img(img, dpi=150, title=None):
    fig, ax = plt.subplots(nrows=1, ncols=1, dpi=dpi)
    ax.set_title( title, fontsize=16, color='black')
    ax.axis('off')
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    return fig, ax

def show_imgs(imgs_dict:dict, ncol=0, dpi=200, font_scale=0.7):
    font_size = int(plt.rcParams["font.size"]*font_scale)

    if ncol > 0:
        nrow = ((len(imgs_dict)-1)//ncol)+1
    else:
        nrow = 1
        ncol = len(imgs_dict)

    img_num = len(imgs_dict)
    fig = plt.figure(figsize=(float(img_num), float(img_num)), dpi=dpi)
    grid = ImageGrid(fig, 111, nrows_ncols=(nrow, ncol), axes_pad=0.2,)

    for i in range(nrow*ncol):
        grid[i].axis('off')
        if i < len(imgs_dict):
            img_key = list(imgs_dict.keys())[i]
            grid[i].imshow(cv2.cvtColor(imgs_dict[img_key], cv2.COLOR_BGR2RGB))
            grid[i].set_title(img_key, fontsize=font_size, color='black', pad=int(font_size/2))
    
    return None

def get_annotation_image(coco:COCO_dataset, imgId):
    # アノテーション描画の設定
    CONFIG = VisConfig()
    CONFIG.ANNO_BBOX_VISUALIZE     = True
    CONFIG.ANNO_KEYPOINT_VISUALIZE = True
    CONFIG.ANNO_MASK_VISUALIZE     = True

    CONFIG.ANNO_BBOX_LINE_SIZE     = 1.5
    CONFIG.MASK_COLORFUL           = True
    CONFIG.ANNO_MASK_LINE_WIDTH    = 4.0

    image_vis = coco.vis_image_matplot(imgId=imgId, plain=False, config=CONFIG)
    ann_img = image_vis.draw_annotations()

    return ann_img

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
coco_old = COCO_dataset(json_path="./sentan_dataset01/sentan_dataset01.json")

coco = COCO_dataset(json_path="./sentan_dataset/sentan_dataset/sentan_dataset.json", dataset_dir="./sentan_dataset/sentan_dataset/", empty_dataset=True)
coco += coco_old
coco.remove_zero_annotation_image()

# other
other_dir = Path("./sentan_dataset/other/")
other_dir.mkdir(exist_ok=True)

old_data = [i.name for i in Path(coco_old.dataset_dir).glob("*.jpg")]
new_data = [i.name for i in Path(coco.dataset_dir).glob("*.jpg")]
diff_data = list(set(old_data) - set(new_data))
for img_name in diff_data:
    shutil.copy(src=Path(coco_old.dataset_dir)/img_name, dst=other_dir/img_name)

# visualized
thumb_dir = Path("./sentan_dataset/visualized/")
thumb_dir.mkdir(exist_ok=True)
for imgId in coco.get_imgId_list():
    ann_img = get_annotation_image(coco, imgId)
    cv2.imwrite(str(thumb_dir/coco.image(imgId).imgInfo["file_name"]), ann_img)

print(coco)

load json file : <./sentan_dataset01/sentan_dataset01.json>
load json file : <./sentan_dataset/sentan_dataset/sentan_dataset.json>


add: 100%|██████████| 11/11 [00:00<00:00, 4206.16it/s]


==========情報==========================================
JSONファイル     	: ./sentan_dataset/sentan_dataset/sentan_dataset.json 
データセット     	: ./sentan_dataset/sentan_dataset/ 
画像の枚数       	: 9/9 (jsonファイル内/実際の枚数)
画質            	: ['960 x 1280'] 
カテゴリー       	: [(1, 'grape_leaf')] 
アノテーション数 	: 70 
アノテーター     	: ['hirahara', 'suzuki', 'Kotaro', 'Ogasawara'] 
READ_ONLY       	: False 
